In [ ]:
# default_exp preprocessing

# Preprocessing data

> Inspecting any particular irregularities and general preparation of the data for modelling.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#export
import pandas as pd
from pathlib import Path
import os
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import typing
import pickle

from sklearn import linear_model, tree, model_selection, ensemble
from ashrae import inspection
from fastai.tabular.all import *

import tqdm

from sklearn import linear_model, tree, model_selection, ensemble

In [ ]:
pd.options.plotting.backend = "plotly"

In [ ]:
data_path = Path("../data")

In [ ]:
csvs = inspection.get_csvs(data_path)
csvs

In [ ]:
%%time
train = inspection.get_core_Xy(csvs['train'])
display(train.head(), train.info())

In [ ]:
%%time
test = inspection.get_core_Xy(csvs['test'])
display(test.head(), test.info())

In [ ]:
%%time
building = inspection.get_building_X(csvs['building'])
display(building.head(), building.info())

In [ ]:
%%time
weather_train = inspection.get_weather_X(csvs['weather_train'])
display(weather_train.head(), weather_train.info())

In [ ]:
%%time
weather_test = inspection.get_weather_X(csvs['weather_test'])
display(weather_test.head(), weather_test.info())

In [ ]:
#export
DEP_VAR = 'meter_reading'
TIME_COL = 'timestamp'

class Processor:
    
    dep_var_stats:dict = None
    
    def __call__(self, df_core:pd.DataFrame, df_building:pd.DataFrame=None,
                 df_weather:pd.DataFrame=None, dep_var:str=None, time_col:str=None,
                 add_time_features:bool=False, add_dep_var_stats:bool=False) -> pd.DataFrame:
    
        # TODO: 
        # - add daily features: temperature delta per site_id, total rain fall, ...
        # - add global stats: mean, median and so on of dep_var by building_id or type
        # - add consumption of the days around the day of interest


        # sanity check presence of df_building if df_weather is given
        if df_weather is not None:
            assert df_building is not None, 'To join the weather info in `df_weather` you need to pass `df_building`.'

        self.dep_var = DEP_VAR if dep_var is None else dep_var
        self.time_col = TIME_COL if time_col is None else time_col

        self.conts, self.cats = [], []

        # sanity check if `df` is a test set (dep_var is missing)
        self.is_train = self.dep_var in df_core.columns

        # core pieces of dependent and independent variables
        self.dep_var_new = f'{self.dep_var}_log1p'
        if self.is_train:
            df_core[self.dep_var_new] = np.log(df_core[self.dep_var].values + 1)
        self.cats += ['building_id', 'meter']

        # adding basic statistics as features
        if add_dep_var_stats:
            df_core = self.add_dep_var_stats(df_core)

        # adding building information
        if df_building is not None:
            df_core = self.add_building_features(df_core, df_building)

        # adding weather information
        if df_weather is not None:
            df_core = self.add_weather_features(df_core, df_weather)
        
        # add timestamp related fields
        if add_time_features:
            df_core = self.add_time_features(df_core)
        
        df_core, var_names = self.cleanup(df_core)
        return df_core, var_names
    
    
    def add_dep_var_stats(self, df_core:pd.DataFrame):
        assert self.is_train or self.dep_var_stats is not None
        if self.is_train: 
            self.dep_var_stats = dict()
        funs = {
            'median': lambda x: torch.median(tensor(x)).item(),
            'mean': lambda x: torch.mean(tensor(x)).item(),
            '5%': lambda x: np.percentile(x, 5),
            '95%': lambda x: np.percentile(x, 95),
        }
        for name, fun in funs.items():
            name = f'{self.dep_var}_{name}'
            self.conts.append(name)
            
            if self.is_train:
                value = fun(df_core[self.dep_var].values)
                df_core[name] = value
                self.dep_var_stats[name] = value
            else:
                df_core[name] = self.dep_var_stats[name]
        return df_core
                
    def add_time_features(self, df_core:pd.DataFrame):
        self.cats.extend(['timestampMonth', 'timestampDay', 'timestampWeek', 'timestampDayofweek',
                     'timestampDayofyear', 'timestampIs_month_end', 'timestampIs_month_start',
                     'timestampIs_quarter_start', 'timestampIs_quarter_end',
                     'timestampIs_year_start', 'timestampIs_year_end',])
        return add_datepart(df_core, self.time_col)
    
    def add_building_features(self, df_core:pd.DataFrame, df_building:pd.DataFrame):
        n = len(df_core)
        df_core = pd.merge(df_core, df_building, on='building_id', how='left')
        assert n == len(df_core)

        self.cats.extend(['site_id', 'primary_use'])
        self.conts.extend(['square_feet', 'year_built', 'floor_count'])
        return df_core
    
    def add_weather_features(self, df_core:pd.DataFrame, df_weather:pd.DataFrame):
        n = len(df_core)
        df_core = pd.merge(df_core, df_weather, on=['site_id', 'timestamp'], how='left')
        assert n == len(df_core)

        self.cats.extend(['cloud_coverage', 'wind_direction'])
        self.conts.extend(['air_temperature', 'dew_temperature', 'precip_depth_1_hr',
                      'sea_level_pressure', 'wind_speed'])
        return df_core
    
    def cleanup(self, df_core:pd.DataFrame):
        # converting cats to category type
        for col in self.cats:
            df_core[col] = df_core[col].astype('category')

        # removing features 
        to_remove_cols = [self.dep_var, 'timestampElapsed', 'timestampYear', self.time_col]
        df_core = df_core.drop(columns=[c for c in df_core.columns if c in to_remove_cols])
        df_core = df_shrink(df_core, int2uint=True)

        var_names = {'conts': self.conts, 'cats': self.cats, 'dep_var': self.dep_var_new}
        if not self.is_train:
            df_core.set_index('row_id', inplace=True)
        missing_cols = [col for col in df_core.columns.values if col not in self.cats + self.conts + [self.dep_var_new]]
        assert len(missing_cols) == 0, f'Missed to assign columns: {missing_cols} to `conts` or `cats`'
        return df_core, var_names

In [ ]:
process_config = dict(
    add_time_features = True,
    add_dep_var_stats = True,
    df_building = building,
    df_weather = weather_train
)
process = Processor()

In [ ]:
%%time
df, var_names = process(train.copy(), 
                        **process_config)

In [ ]:
%%time
df_test, _ = process(test.copy(), 
                                  **process_config)

In [ ]:
#hide
assert len(df_test.columns) + 1 == len(df.columns)

In [ ]:
df_test.head().T

In [ ]:
df_test.info()

In [ ]:
#export
def test_var_names(var_names:dict):
    assert isinstance(var_names, dict)
    assert 'conts' in var_names and 'cats' in var_names and 'dep_var' in var_names
    assert isinstance(var_names['conts'], list) 
    assert isinstance(var_names['cats'], list) 
    assert isinstance(var_names['dep_var'], str)

In [ ]:
test_var_names(var_names)

In [ ]:
#export
def store_var_names(data_path:Path, var_names:dict):
    fname = data_path/'var_names.pckl'
    print(f'Storing var names at: {fname}')
    with open(fname, 'wb') as f:
        pickle.dump(var_names, f)

In [ ]:
%%time
store_var_names(data_path, var_names)

In [ ]:
#export
def load_var_names(fname:Path):
    print(f'Reading var names at: {fname}')
    with open(fname, 'rb') as f:
        var_names = pickle.load(f)
    return var_names

In [ ]:
%%time
# var_names = load_var_names(data_path/'var_names.pckl')

In [ ]:
#hide
test_var_names(var_names)

In [ ]:
#export
def store_df(path:Path, df:pd.DataFrame): df.to_parquet(path)

In [ ]:
%%time
# store_df(data_path/'X.parquet', df)

In [ ]:
%%time
# store_df(data_path/'X_test.parquet', df_test)

In [ ]:
#export
def load_df(path:Path): return pd.read_parquet(path)

In [ ]:
%%time
# df = load_df(data_path/'X.parquet')

Training to get a basic idea if the added features do have any benefit

In [ ]:
def train_predict(df:pd.DataFrame, var_names:dict, 
                  model, params:dict=None, n_rep:int=3,
                  n_samples_train:int=10000, 
                  n_samples_test:int=10000,
                  test_size:float=.2):

    y_col = var_names['dep_var']
    score_vals = []
    params = {} if params is None else params

    procs = [Categorify, FillMissing, Normalize]
    to = TabularPandas(df.copy(), procs, 
                       var_names['cats'], var_names['conts'], 
                       y_names=var_names['dep_var'])
    
    for i in tqdm.tqdm(range(n_rep), total=n_rep, desc='Repetition'):
        
        mask = to.xs.index.isin(
            np.random.choice(to.xs.index.values, size=int(test_size*len(to.xs)), replace=False)
        )
        
        m = model(**params)
        
        _X = to.xs.loc[~mask, :].iloc[:n_samples_train]
        _y = to.ys.loc[~mask, y_col].iloc[:n_samples_train]
        m.fit(_X.values, _y.values)
        
        _X = to.xs.loc[mask, :].iloc[:n_samples_test]
        _y = to.ys.loc[mask, y_col].iloc[:n_samples_test]
        pred = m.predict(_X.values)
        s = torch.sqrt(F.mse_loss(tensor(pred), tensor(_y.values))).item()
        score_vals.append({'iter': i, 'rmse loss': s})
    
    return pd.DataFrame(score_vals)

In [ ]:
params = {'n_estimators': 20, 'max_features': 'sqrt'}
model = ensemble.RandomForestRegressor
# params = None
# model = linear_model.LinearRegression
n_rep = 21
n_samples_train = 10000
n_samples_test = 1000

In [ ]:
%%time
df_rep = train_predict(df.copy(), var_names, model, params=params, 
                       n_rep=n_rep, n_samples_train=n_samples_train,
                       n_samples_test=n_samples_test)

In [ ]:
df_rep['rmse loss'].describe()

In [ ]:
px.box(df_rep, y='rmse loss', range_y=(0, 2.5))

Baseline model = RandomForest with 20 estimators and sqrt features, training over 100k samples and predicting over 1k

<table>
    <tr>
        <th>input</th>
        <th>rmse loss</th>
        <th>time [s/it]</th>
    </tr>
    <tr>
        <td>meter and building id only</td>
        <td>1.2 - 1.21</td>
        <td>10.2</td>
    </tr>
    <tr>
        <td>using dep_var stats</td>
        <td>1.16 - 1.18</td>
        <td>17.3</td>
    </tr>
    <tr>
        <td>using time stats</td>
        <td>1.2 - 1.21</td>
        <td>13.2 - 13.7</td>
    </tr>
    <tr>
        <td>using building info</td>
        <td>1.19</td>
        <td>17 - 18</td>
    </tr>
    <tr>
        <td>using weather (+ building) info</td>
        <td>1.13 - 1.139</td>
        <td>14.6 - 15</td>
    </tr>
    <tr>
        <td>using all above</td>
        <td>1.19 - 1.21</td>
        <td>20 - 26</td>
    </tr>
</table>

## old stuff

In [ ]:
cols = var_names['conts'] + var_names['cats']
y_col = var_names['dep_var']
test_size = .2

In [ ]:
%%time
X_train, X_test, y_train, y_test = model_selection.train_test_split(df.loc[:,cols].values, df[y_col].values,
                                                                            test_size=test_size)

In [ ]:
%%time
mask = df.index.isin(np.random.choice(df.index.values, size=int(test_size*len(df)), replace=False))

In [ ]:
base_path = Path("../data")

In [ ]:
csvs = sorted([base_path/v for v in os.listdir(base_path) if v.endswith('.csv')])
csvs

In [ ]:
train_csv = csvs[3]
train_weather_csv = csvs[-1]
test_csv = csvs[2]
test_weather_csv = csvs[-2]
meta_csv = csvs[0]

train_csv, train_weather_csv, test_csv, test_weather_csv, meta_csv

## Loading

In [ ]:
%%time
train = pd.read_csv(train_csv, parse_dates=['timestamp'])
train.head()

In [ ]:
%%time
test = pd.read_csv(test_csv, parse_dates=['timestamp'])
test.head()

In [ ]:
%%time
weather_train = pd.read_csv(train_weather_csv, parse_dates=['timestamp'])
weather_train.head()

In [ ]:
%%time
weather_test = pd.read_csv(test_weather_csv, parse_dates=['timestamp'])
weather_test.head()

In [ ]:
%%time
building = pd.read_csv(meta_csv)
building.head()

## Inspection of the data

In [ ]:
train.head()

Kicking out outlying measurements

In [ ]:
%%time
train_meter_stats = (train.groupby(['meter'])['meter_reading']
                     .describe(percentiles=[.05, .25, .5, .75, .95]))
train_meter_stats

In [ ]:
%%time
mask = pd.concat([(train['meter']==m) & (train['meter_reading'] < 1e2*grp['95%'].iloc[0])
         for m, grp in train_meter_stats.groupby(['meter'])], axis=1).any(axis=1)
mask

In [ ]:
print(f'removing {(1 - mask.sum()/len(mask)) * 100:.3f} % of the data')
print(f'min {train.loc[~mask, "meter_reading"].min()}, max {train.loc[~mask, "meter_reading"].max()}')

In [ ]:
train = train.loc[mask]

Looking into time series with a lot of 0s

In [ ]:
%%time
train_meter_stats = (train.groupby(['meter', 'building_id'])['meter_reading']
                     .describe(percentiles=[.05, .25, .5, .75, .95]))
train_meter_stats

In [ ]:
with pd.option_context('display.max_rows',200):
    display(train_meter_stats.loc[train_meter_stats['50%'] == 0])

In [ ]:
train.head()

In [ ]:
%%time
bid = 112
meter = 3
mask = (train['building_id']==bid) & (train['meter']==meter)
print(f'number of observations: {mask.sum()}')

In [ ]:
%%time
train.loc[mask].plot(x='timestamp', y='meter_reading')

Finding:
* electricity: meter 0 should probably never be close to 0
* chilledwater: meter 1 can be continuously 0 at night
* steam: meter 2 should probably never be close to 0
* hotwater: meter 3 possibly also shouldn't be 0

Dropping 0s for electricity readings

In [ ]:
meter_maps = {0: 'electricity', 1: 'chilledwater', 2: 'steam', 3: 'hotwater'}
inv_meter_maps = {v:k for k,v in meter_maps.items()}

In [ ]:
mask = (train['meter'] == inv_meter_maps['electricity']) & (train['meter_reading'] > 0) \
    | (train['meter'] != inv_meter_maps['electricity'])
# mask = ~np.isclose(train['meter_reading'], 0)
print(f'removing {(1 - mask.sum()/len(mask)) * 100:.3f} % of the data')
print(f'min {train.loc[~mask, "meter_reading"].min()}, max {train.loc[~mask, "meter_reading"].max()}')

In [ ]:
# train.loc[~mask, 'meter_reading'] = np.nan
train = train.loc[mask]

A broader look at the distribution of `meter_reading` given `meter`

In [ ]:
s = train.loc[train['meter_reading']<20000].groupby('meter').sample(n=5000)
px.histogram(s, x='meter_reading', color='meter',
             barmode='overlay', opacity=.5,
             histnorm='probability density')

In [ ]:
nbins = 1000
all_paretos = []
for meter, grp in train.groupby('meter'):
    mask = train['meter']==meter
    max_val = train.loc[mask,'meter_reading'].max()
    bins = np.logspace(0, np.log10(max_val), nbins)

    s = (pd.cut(train.loc[mask, 'meter_reading'].sample(5000), 
                bins=bins)
         .value_counts()
         .sort_index()
         .to_frame()
         .rename(columns={'meter_reading': 'count'}))
    
    s['pareto share'] = s['count'].cumsum() / s['count'].sum()
    s['x'] = [v.right for v in s.index.values]
    s['log(1+x)'] = [np.log(1+v.right) for v in s.index.values]
    s['meter'] = meter_maps[meter]
    
    print('\n', meter_maps[meter])
    display(s.loc[s['pareto share'] < .99, ['pareto share']])
    all_paretos.append(s)

In [ ]:
all_paretos = pd.concat(all_paretos, ignore_index=True)

In [ ]:
px.line(all_paretos, x='log(1+x)', y='pareto share', color='meter')

In [ ]:
px.line(all_paretos, x='x', y='pareto share', color='meter')

## Radically merging all the data

In [ ]:
#export
def radical_merging(df:pd.DataFrame, building:pd.DataFrame, 
                    weather:pd.DataFrame, n_sample:int=None,
                    training:bool=True):
    
    tmp = df.copy(deep=True)

    bid_col = 'building_id'
    sid_col = 'site_id'
    time_col = 'timestamp'
    target_col = 'meter_reading'
    
    categorical = ['meter', 'primary_use', 'cloud_coverage', bid_col, sid_col]
    continuous = ['square_feet', 'year_built', 'floor_count', 
                  'air_temperature', 'dew_temperature',
                  'precip_depth_1_hr', 'sea_level_pressure', 'wind_direction',
                  'wind_speed']

    x_cols = [bid_col, 'meter', target_col, time_col] if training \
            else [bid_col, 'meter', time_col]
    X = tmp.loc[:,x_cols].copy()

    X = pd.merge(X, building, on=bid_col, how='left')
    X = pd.merge(X, weather, on=[sid_col, time_col], how='left')

    #return_cols =  categorical + continuous + [target_col,]  # time_col

    #X = X.loc[:,return_cols]
    if n_sample is not None:
        X = X.sample(n_sample)
        
    if training:
        X[target_col] = np.log(X[target_col] + 1)
        
    X = add_datepart(X, time_col)
    categorical.extend(['timestampMonth', 'timestampWeek', 'timestampDay',
                        'timestampDayofweek', 'timestampDayofyear', 'timestampIs_month_end',
                        'timestampIs_month_start', 'timestampIs_quarter_end',
                        'timestampIs_quarter_start', 'timestampIs_year_end',
                        'timestampIs_year_start'])
    
    continuous.extend(['timestampYear', 'timestampElapsed'])
        
    X = X.loc[:, [col for col in X.columns.values if col not in [time_col]]]
    
    missing_cont = [col for col in continuous if col not in X.columns]
    missing_cat = [col for col in categorical if col not in X.columns]
    assert len(missing_cat) == 0, f'{missing_cat} not in X!'
    assert len(missing_cont) == 0, f'{missing_cont} not in X!'
    
    X.loc[:,continuous] = X.loc[:,continuous].astype(float)
    X.loc[:,categorical] = X.loc[:,categorical].astype('category')
    
    return X, continuous, categorical

Generating train / validate features

In [ ]:
%%time
n_sample = None  #10000
X, continuous, categorical = radical_merging(train.copy(), building, weather_train,
                    n_sample=n_sample)

In [ ]:
%%time
X.to_parquet('../data/X.parquet')

Storing variable types

In [ ]:
with open('../data/var_types.pckl', 'wb') as f:
    pickle.dump({'cont':continuous, 'cat':categorical}, f)

Generating test set features

In [ ]:
%%time
X_test, _, _ = radical_merging(test.copy(), building, weather_test,
                    n_sample=None, training=False)

In [ ]:
X_test.to_parquet('../data/X_test.parquet')